# Recreating coniii mft

In [1]:
from clean_data import voting_data as samples

In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize_scalar

In [3]:
samples = (np.array(samples) + 1.0)/2.0

In [4]:
n = len(samples[0])
N = len(samples)

In [13]:
def cooccurrence_matrix(samples):
    """ Takes the DATA samples, returns the cooccurrence matrix. The samples must be in the {0, 1} 
    basis. The cooccurrence matrix is the """
    return(samples.T @ samples / len(samples))

def replaceDiag(mat, lst):
    """ Replaces the diagonal of a matrix mat with lst """
    return(mat - sp.diag(sp.diag(mat)) + sp.diag(lst))

def get_regularized_MF_J(samples, mean_field_prior_lambda):
    cooc = cooccurrence_matrix(samples)
    freqs = sp.diag(cooc) # These are the pi, pj
    c = cooc - sp.outer(freqs, freqs) # This is pij - pi pj
    
    Mdenom = sp.sqrt(sp.outer(freqs * (1.-freqs), freqs * (1. - freqs)))
    M = c / Mdenom # This is just the correlations.

    # Calculating off diagonal J elements
    #mean_field_prior_lambda = 0
    gamma = mean_field_prior_lambda / len(samples)
    mq, vq = la.eig(M)
    mqhat = 0.5*(mq - gamma + sp.sqrt((mq - gamma)**2 + 4. * gamma))
    jq = 1.0 / mqhat
    Jprime = sp.real_if_close(sp.dot(vq, sp.dot(sp.diag(jq) , vq.T)))
    JMF = replaceDiag(Jprime / Mdenom, sp.zeros(n))
    
    # Diagonal J elements -- can we not just set these to 0?
    piFactor = sp.repeat( [(freqs-0.5)/(freqs*(1.-freqs))], n, axis=0).T
    pjFactor = sp.repeat([freqs], n, axis=0)
    factor2 = c * piFactor - pjFactor
    hMF = sp.diag( sp.dot(JMF, factor2.T)).copy()
    hMF -= sp.log(freqs/(1.-freqs))

    J = replaceDiag(0.5*JMF, hMF)
    
    return(J)

In [14]:
cooc = cooccurrence_matrix(samples)

They're using an upper triangular cooccurrence matrix. There's also a cluster term, but they take the cluster to be the length o the coocurrence matrix. I think this means that they just reused code from the cluster expansion (which looks awesome, and you should learn about it) and set the cluster to be the full length of all the spins. For mean field theory, they stop using an upper triangular matrix, so we don't need to do that.

In [15]:
def cooc_stdevs(cooc_matrix, num_samples):
    """ Returns a list of variances of the samples using Laplace's Method (rule of succession)"""
    coocBayesianMean = (cooc*num_samples + 1.0) / (2.0 + num_samples) # Laplace's Rule
    return(sp.sqrt(coocBayesianMean * (1.-coocBayesianMean)/num_samples))

In [16]:
cooc_stdev = cooc_stdevs(cooc, N)

This is used to find the upper triangular indices so we don't count everything twice. 
a[np.triu_indices(3, k = 0 or 1)]

In [17]:
freqsList = np.diag(cooc)
pmean = np.mean(freqsList)
N = len(samples)
n = len(cooc)

In [18]:
def get_samples(J, num_samples = 20*N, T = 2.0):
    """ Wrapper function to obtain the samples """
    start_state = np.random.choice([-1,1], size=n)
    ising_samples = fast.cfast_sample(start_state, J, num_samples = num_samples, T=T)
    return(ising_samples)

In [19]:
# I don't think we'll need to use this.
def convert_params(J, convert_to, concat=False):
    """Convert Ising model fields and couplings from {0,1} basis to {-1,1} and vice versa. convert_to
    should be either '01' or '11'
    """
    if convert_to == '11':
        Jp = J / 4.
    elif convert_to == '01':
        Jp = J * 4.

    return([hp, Jp])

In [20]:
import fast

In [22]:
%%time
J_ = get_regularized_MF_J(samples,0.0)

CPU times: user 2.64 ms, sys: 554 µs, total: 3.19 ms
Wall time: 2.11 ms


In [ ]:
%%time
get_samples(J_, T = 1.0)

In [16]:
cooc= cooccurrence_matrix(samples)

In [12]:
J = np.loadtxt("jij_correct_solution.csv", delimiter=",")

In [ ]:
import scipy

In [67]:
J_ = get_regularized_MF_J(samples,0.0)

In [68]:
a = get_samples(J_, num_samples=16000, T=10)

In [57]:
def minimizer_func(mean_field_prior_gamma):
    """meanFieldGamma is the strength of the Ising problem. We translate ot the QUBO problem,
    extract the coocurrence matrix of the samples, and compare it to the coocurrence matrix of
    the data. """
    mean_field_prior_lambda = mean_field_prior_gamma / (pmean**2 * (1.-pmean)**2) # I don't know where this comes from
    J = get_regularized_MF_J(cooc, mean_field_prior_lambda)
    ising_samples = get_samples(J, T = 10)[1]
    print("Acquired samples")
    qubo_samples = (ising_samples + 1.0) / 2.0
    
    sample_cooc = cooccurrence_matrix(qubo_samples)
    diff = (cooc - sample_cooc) / cooc_stdev
    dc = diff[np.triu_indices(n)]
    return(sp.sum(dc ** 2))

In [48]:
def convert_solution_to_ising(gamma_sol):
    mean_field_lambda = gamma_sol / (pmean**2 * (1.-pmean)**2)
    J = get_regularized_MF_J(cooc, mean_field_lambda, N)
    J = J + J.T
    # convert J to {-1,1} basis
    h = np.diag(-J)
    J = J - np.diag(np.diag(J))
    final_J = convert_params(h, J*2, '11')
    return(final_J)

In [72]:
a = get_samples(J_, num_samples=16000, T=10)

KeyboardInterrupt: 

In [71]:
J_ = get_regularized_MF_J(cooc, 0.0)